# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `mongoimport --db uk_food --collection establishments --file`

In [13]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint


In [14]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [15]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'classroom', 'config', 'epa', 'local', 'met', 'test', 'uk_food']


In [16]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [17]:
# review the collections in our new database
print(db.list_collection_names())

['establishments']


In [18]:
# review a document in the establishments collection
pprint(db.establishements.find_one())

None


In [19]:
# assign the collection to a variable
establishments = db['establishments']
print(establishments.find_one())

{'_id': ObjectId('64f7aa449a469dd57963437f'), 'establishments': 'Penang Flavors'}


## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [20]:
# Create a dictionary for the new restaurant data
rest_data_dic = {}

In [21]:
# Insert the new restaurant into the collection
penang_new = establishments.insert_one({'establishments' : 'Penang Flavors'})

In [22]:
# Check that the new restaurant was inserted
print(penang_new)

2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [23]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields

result = establishments.find_one(
    {"BusinessTypeName": "Restaurant/Cafe/Canteen"},
    {"BusinessTypeID": 1, "BusinessTypeName": 1, "_id": 0}
)

print(result)

None


3. Update the new restaurant with the `BusinessTypeID` you found.

In [24]:
# Update the new restaurant with the correct BusinessTypeID

new_business_type = {
    'BusinessTypeID': result['BusinessTypeID'],
    'BusinessType': result['BusinessTypeName']
}

filter = {"BusinessName": "New Restaurant"}

update = {"$set": new_business_type}

# Update the document with the new BusinessTypeID and BusinessType
new_update = establishments.update_one(filter, update)

print(new_update.modified_count, "documents updated.")

TypeError: 'NoneType' object is not subscriptable

In [ ]:
# Confirm that the new restaurant was updated
print(new_update.modified_count, "documents updated.")

4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [ ]:
# Find how many documents have LocalAuthorityName as "Dover"
dover = establishments.count_documents({"LocalAuthorityName": "Dover"})
print(dover)

In [ ]:
# Delete all documents where LocalAuthorityName is "Dover"
delete_dover = establishments.delete_many({"LocalAuthorityName": "Dover"})

In [ ]:
# Check if any remaining documents include Dover
dover_check= establishments.find_one({"LocalAuthorityName": "Dover"})
print(dover_check)

In [ ]:
# Check that other documents remain with 'find_one'
if result is not None:
    remaining_docs = establishments.find({"LocalAuthorityName": "Dover"})
    for doc in remaining_docs:
        print(doc)
else:
    print("No documents found.")

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [ ]:
# Change the data type from String to Decimal for longitude and latitude
data_change = {
    "$set": {
        "latitude": { "$toDouble": "$latitude" },
        "longitude": { "$toDouble": "$longitude" }
    }
}

Use `update_many` to convert `RatingValue` to integer numbers.

In [ ]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [ ]:
# Change the data type from String to Integer for RatingValue
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])
establishments.update_many({"RatingValue": {"$type": 2}}, [ {'$set':{ "RatingValue" : { "$toInt": "$RatingValue" } }} ])

In [ ]:
# Check that the coordinates and rating value are now numbers
result = establishments.find_one({"BusinessName": "New Restaurant"})

print("Longitude (type): ", type(result["geocode"]["longitude"]))
print("Latitude (type): ", type(result["geocode"]["latitude"]))
print("RatingValue (type): ", type(result["RatingValue"]))